In [2]:
# import json
# import logging
# import os

# import joblib
# import pandas as pd
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
# from sklearn.naive_bayes import MultinomialNB
# from spellchecker import SpellChecker

# logging.basicConfig(
#     level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
# )


# def load_labeled_data(filename):
#     """Load labeled data from a JSON file and return it as a DataFrame."""
#     try:
#         with open(filename, "r", encoding="utf-8") as file:
#             data = json.load(file)

#         # Ensure 'data' and 'label' keys are present and non-empty in each item
#         labeled_data = [
#             (item["data"], item["label"])
#             for item in data
#             if "data" in item and "label" in item and item["data"] and item["label"]
#         ]
#         if not labeled_data:
#             raise ValueError("No valid labeled data found.")
#         return pd.DataFrame(labeled_data, columns=["data", "labels"])
#     except (FileNotFoundError, json.JSONDecodeError, ValueError) as e:
#         print(f"Error loading data: {e}")
#         return pd.DataFrame(columns=["data", "labels"])


# def load_list_from_json(filename):
#     """Load a list of strings from a JSON file."""
#     if os.path.exists(filename):
#         with open(filename, "r", encoding="utf-8") as file:
#             return set(json.load(file))
#     return set()


# def save_list_to_json(filename, data_list):
#     """Save a list of strings to a JSON file."""
#     with open(filename, "w", encoding="utf-8") as file:
#         json.dump(list(data_list), file)


# def check_spelling(spell, words_to_check):
#     """Check spelling and get suggestions for a list of words."""
#     corrected_words = [spell.correction(word) for word in words_to_check]
#     suggestions = [list(spell.candidates(word)) for word in words_to_check]
#     return corrected_words, suggestions


# def fit_vectorizer(X_train, vectorizer=None):
#     """Fit or update a CountVectorizer for character n-grams."""
#     if vectorizer is None:
#         vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
#     vectorizer.fit(X_train)
#     return vectorizer


# def train_model(X_train_vectorized, y_train):
#     """Train a Naive Bayes model."""
#     model = MultinomialNB()
#     model.fit(X_train_vectorized, y_train)
#     return model



# def main():
#     logging.info("Starting the main process.")
#     labeled_data_filename = "data_output.json"
#     custom_dictionary_filename = "custom_dictionary.json"
#     misclassified_samples_filename = "misclassified_samples.json"
#     model_filename = "naive_bayes_model.pkl"

#     spell = SpellChecker(language="fr")

#     # Load custom dictionary and misclassified samples as sets
#     custom_dictionary = load_list_from_json(custom_dictionary_filename)
#     spell.word_frequency.load_words(custom_dictionary)

#     misclassified_samples = load_list_from_json(misclassified_samples_filename)

#     misclassification_threshold = 5
#     epochs = 0

#     all_X_train = pd.Series(dtype=str)
#     all_y_train = pd.Series(dtype=int)

#     while epochs < 20:
#         labeled_df = load_labeled_data(labeled_data_filename)
#         if (
#             labeled_df.empty
#             or "labels" not in labeled_df.columns
#             or "data" not in labeled_df.columns
#         ):
#             print("Error: Labeled data is empty or missing required columns.")
#             break

#         X_train, X_test, y_train, y_test = train_test_split(
#             labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
#         )

#         all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
#         all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

#         vectorizer = fit_vectorizer(all_X_train)
#         X_train_vectorized = vectorizer.transform(all_X_train)
#         model = train_model(X_train_vectorized, all_y_train)

#         # Find the minimum number of samples in any class
#         min_class_size = min(
#             [sum(all_y_train == class_label) for class_label in set(all_y_train)]
#         )

#         # Set n_splits to the minimum of 3 or the smallest class size
#         n_splits = min(3, min_class_size)

#         if n_splits > 1:
#             cv = StratifiedKFold(n_splits=n_splits)
#             cv_scores = cross_val_score(model, X_train_vectorized, all_y_train, cv=cv)
#             print(
#                 f"Cross-validation scores: {cv_scores}, Mean score: {cv_scores.mean()}"
#             )
#         else:
#             print("Not enough data to perform cross-validation.")

#         words_to_check = ["Bore", "Lre"]
#         predictions = model.predict(vectorizer.transform(words_to_check))

#         new_misclassified = [
#             word
#             for word, prediction in zip(words_to_check, predictions)
#             if prediction == 0
#         ]
#         misclassified_samples.update(new_misclassified)

#         if len(misclassified_samples) >= misclassification_threshold:
#             break

#         epochs += 1

#     # Save the trained model
#     joblib.dump(model, model_filename)

#     # Save the updated custom dictionary
#     new_vocabulary = set(vectorizer.get_feature_names_out())
#     custom_dictionary.update(new_vocabulary)
#     save_list_to_json(custom_dictionary_filename, custom_dictionary)

#     # Save misclassified samples
#     save_list_to_json(misclassified_samples_filename, misclassified_samples)

#     # Check spelling and print corrections and suggestions
#     corrected_words, suggestions = check_spelling(spell, words_to_check)
#     for word, corrected, suggestion_list in zip(
#         words_to_check, corrected_words, suggestions
#     ):
#         print(
#             f"Original: {word}, Corrected: {corrected}, Suggestions: {suggestion_list}"
#         )

#     print("Training loop finished.")


# if __name__ == "__main__":
#     main()

In [20]:
# import json
# import logging
# import os
# import joblib
# import pandas as pd
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
# from sklearn.naive_bayes import MultinomialNB
# from spellchecker import SpellChecker

# logging.basicConfig(
#     level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
# )


# def load_labeled_data(filename):
#     """Load labeled data from a JSON file and return it as a DataFrame."""
#     try:
#         with open(filename, "r", encoding="utf-8") as file:
#             data = json.load(file)

#         # Ensure 'data' and 'label' keys are present and non-empty in each item
#         labeled_data = [
#             (item["data"], item["label"])
#             for item in data
#             if "data" in item and "label" in item and item["data"] and item["label"]
#         ]
#         if not labeled_data:
#             raise ValueError("No valid labeled data found.")
#         return pd.DataFrame(labeled_data, columns=["data", "labels"])
#     except (FileNotFoundError, json.JSONDecodeError, ValueError) as e:
#         logging.error(f"Error loading data: {e}")
#         return pd.DataFrame(columns=["data", "labels"])


# def load_list_from_json(filename):
#     """Load a list of strings from a JSON file."""
#     if os.path.exists(filename):
#         with open(filename, "r", encoding="utf-8") as file:
#             return set(json.load(file))
#     return set()


# def save_list_to_json(filename, data_list):
#     """Save a list of strings to a JSON file."""
#     with open(filename, "w", encoding="utf-8") as file:
#         json.dump(list(data_list), file)


# # def check_spelling(spell, words_to_check):
# #     """Check spelling and get suggestions for a list of words."""
# #     corrected_words = [spell.correction(word) for word in words_to_check]
# #     suggestions = [list(spell.candidates(word)) for word in words_to_check]
# #     return corrected_words, suggestions

# def check_spelling(spell, words_to_check):
#     """Check spelling and get suggestions for a list of words."""
#     corrected_words = [spell.correction(word) for word in words_to_check]
#     suggestions = [list(spell.candidates(word)) if spell.candidates(word) is not None else [] for word in words_to_check]
#     return corrected_words, suggestions


# def fit_vectorizer(X_train, vectorizer=None):
#     """Fit or update a CountVectorizer for character n-grams."""
#     if vectorizer is None:
#         vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
#     vectorizer.fit(X_train)
#     return vectorizer


# def train_model(X_train_vectorized, y_train):
#     """Train a Naive Bayes model."""
#     model = MultinomialNB()
#     model.fit(X_train_vectorized, y_train)
#     return model


# def main():
#     logging.info("Starting the main process.")
#     labeled_data_filename = "data_output.json"
#     custom_dictionary_filename = "custom_dictionary.json"
#     misclassified_samples_filename = "misclassified_samples.json"
#     model_filename = "naive_bayes_model.pkl"

#     spell = SpellChecker(language="fr")

#     # Load custom dictionary and misclassified samples as sets
#     custom_dictionary = load_list_from_json(custom_dictionary_filename)
#     spell.word_frequency.load_words(custom_dictionary)

#     misclassified_samples = load_list_from_json(misclassified_samples_filename)

#     misclassification_threshold = 5
#     epochs = 0

#     all_X_train = []  # Use list instead of pd.Series for efficient concatenation
#     all_y_train = []

#     while epochs < 20:
#         labeled_df = load_labeled_data(labeled_data_filename)
#         if (
#             labeled_df.empty
#             or "labels" not in labeled_df.columns
#             or "data" not in labeled_df.columns
#         ):
#             logging.error("Error: Labeled data is empty or missing required columns.")
#             break

#         X_train, X_test, y_train, y_test = train_test_split(
#             labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
#         )

#         all_X_train.extend(X_train)  # Append to the list
#         all_y_train.extend(y_train)  # Append to the list

#         vectorizer = fit_vectorizer(all_X_train)
#         X_train_vectorized = vectorizer.transform(all_X_train)
#         model = train_model(X_train_vectorized, all_y_train)

#         # Find the minimum number of samples in any class
#         min_class_size = min(
#             [
#                 sum(pd.Series(all_y_train) == class_label)
#                 for class_label in set(all_y_train)
#             ]
#         )

#         # Set n_splits to the minimum of 3 or the smallest class size
#         n_splits = min(3, min_class_size)

#         if n_splits > 1:
#             cv = StratifiedKFold(n_splits=n_splits)
#             cv_scores = cross_val_score(model, X_train_vectorized, all_y_train, cv=cv)
#             logging.info(
#                 f"Cross-validation scores: {cv_scores}, Mean score: {cv_scores.mean()}"
#             )
#         else:
#             logging.warning("Not enough data to perform cross-validation.")

#         words_to_check = ["motday", "hgost", "mney", "elphat","databreh","alpioe"]
#         predictions = model.predict(vectorizer.transform(words_to_check))

#         new_misclassified = [
#             word
#             for word, prediction in zip(words_to_check, predictions)
#             if prediction == 0
#         ]
#         misclassified_samples.update(new_misclassified)

#         if len(misclassified_samples) >= misclassification_threshold:
#             break

#         epochs += 1

#     # Save the trained model
#     joblib.dump(model, model_filename)

#     # Save the updated custom dictionary
#     new_vocabulary = set(vectorizer.get_feature_names_out())
#     custom_dictionary.update(new_vocabulary)
#     save_list_to_json(custom_dictionary_filename, custom_dictionary)

#     # Save misclassified samples
#     save_list_to_json(misclassified_samples_filename, misclassified_samples)

#     # Check spelling and print corrections and suggestions
#     corrected_words, suggestions = check_spelling(spell, words_to_check)
#     for word, corrected, suggestion_list in zip(
#         words_to_check, corrected_words, suggestions
#     ):
#         logging.info(
#             f"Original: {word}, Corrected: {corrected}, Suggestions: {suggestion_list}"
#         )

#     logging.info("Training loop finished.")


# if __name__ == "__main__":
#     main()


2024-12-04 10:35:52,116 - INFO - Starting the main process.


2024-12-04 10:35:52,266 - WARNING - Not enough data to perform cross-validation.
2024-12-04 10:35:52,276 - INFO - Cross-validation scores: [1. 1.], Mean score: 1.0
2024-12-04 10:35:52,286 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:35:52,297 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:35:52,310 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:35:52,319 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:35:52,330 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:35:52,341 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:35:52,350 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:35:52,363 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:35:52,376 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:35:52,393 - INFO - Cross-validation scores: [1. 

It looks like your code is running as expected with repeated cross-validation scores of 1.0 and logging a warning when not enough data is available for cross-validation. However, there are a few points to consider or improve:

### Observations:

1. **Repeated Cross-validation Results**: You're seeing repeated `Cross-validation scores: [1. 1. 1.]` with a mean score of `1.0` in each iteration. This suggests that your model is always achieving perfect accuracy on the training set. This could indicate one of the following:

   - **Small Dataset**: If your training dataset is small, the model might memorize the data rather than generalize.
   - **Data Imbalance**: If your classes are imbalanced, the model may predict the majority class, leading to perfect scores on the training fold but poor generalization to unseen data.
   - **Overfitting**: If the dataset is simple or the model too powerful for the data, you might be overfitting.

2. **Cross-Validation Warning**: The warning `Not enough data to perform cross-validation.` is logged, but this doesn't seem to be an issue in the subsequent logs, where cross-validation happens. It could be that `n_splits` is set to `1` due to too few samples of a particular class, but the model still runs cross-validation with a reduced number of splits (e.g., only 1 fold instead of 3).

3. **Spell Checking**: The function `check_spelling` is correctly processing a list of words with spelling mistakes, returning corrections and suggestions.

### Possible Improvements:

1. **Address the Data Imbalance**:

   - Check if your data is imbalanced (some classes may be much larger than others), which might be skewing the results. You could try techniques such as:
     - **Resampling**: Use oversampling (e.g., SMOTE) or undersampling to balance your dataset.
     - **Class Weighting**: Modify the `MultinomialNB` model to account for class imbalance by adjusting class weights.

2. **Ensure Sufficient Data for Cross-Validation**:

   - If you consistently see the warning, it may be useful to check the actual distribution of class sizes in your dataset and adapt the cross-validation approach to handle small class sizes better.

3. **Model Evaluation**:

   - If your model consistently gets perfect scores, it might be overfitting. You should validate the model with a separate validation set or use a hold-out set (test data) to evaluate real-world performance. Cross-validation is great, but it doesn't help much if the model is overfitting to the training data.

4. **Refine Stopping Criteria**:

   - Instead of using the `misclassification_threshold` as the stopping condition for your loop, consider monitoring both performance metrics (accuracy, precision, recall) and the model's ability to generalize across different classes.

5. **Logging and Debugging**:
   - To get more detailed insight into your model's performance, consider logging more information such as confusion matrices, class-wise accuracy, or other classification metrics like F1-score, precision, and recall.

### Updated Code Example (with class balancing and enhanced logging):

Here’s a slightly modified version of your main loop, adding an imbalance correction and more detailed logging.

```python
from sklearn.utils.class_weight import compute_class_weight

def train_model_with_class_weight(X_train_vectorized, y_train):
    """Train a Naive Bayes model with class weights."""
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    weight_dict = dict(zip(np.unique(y_train), class_weights))
    model = MultinomialNB(class_prior=weight_dict)
    model.fit(X_train_vectorized, y_train)
    return model

def main():
    logging.info("Starting the main process.")
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"
    model_filename = "naive_bayes_model.pkl"

    spell = SpellChecker(language="fr")

    # Load custom dictionary and misclassified samples as sets
    custom_dictionary = load_list_from_json(custom_dictionary_filename)
    spell.word_frequency.load_words(custom_dictionary)

    misclassified_samples = load_list_from_json(misclassified_samples_filename)

    misclassification_threshold = 5
    epochs = 0

    all_X_train = []  # Use list instead of pd.Series for efficient concatenation
    all_y_train = []

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if labeled_df.empty or "labels" not in labeled_df.columns or "data" not in labeled_df.columns:
            logging.error("Error: Labeled data is empty or missing required columns.")
            break

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train.extend(X_train)  # Append to the list
        all_y_train.extend(y_train)  # Append to the list

        vectorizer = fit_vectorizer(all_X_train)
        X_train_vectorized = vectorizer.transform(all_X_train)

        # Train model with class weights
        model = train_model_with_class_weight(X_train_vectorized, all_y_train)

        # Cross-validation
        n_splits = min(3, min([sum(pd.Series(all_y_train) == class_label) for class_label in set(all_y_train)]))
        if n_splits > 1:
            cv = StratifiedKFold(n_splits=n_splits)
            cv_scores = cross_val_score(model, X_train_vectorized, all_y_train, cv=cv)
            logging.info(f"Cross-validation scores: {cv_scores}, Mean score: {cv_scores.mean()}")
        else:
            logging.warning("Not enough data to perform cross-validation.")

        # Check for misclassified samples
        words_to_check = ["motday", "hgost", "mney", "elphat", "rmshkana"]
        predictions = model.predict(vectorizer.transform(words_to_check))

        new_misclassified = [
            word for word, prediction in zip(words_to_check, predictions) if prediction == 0
        ]
        misclassified_samples.update(new_misclassified)

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save the trained model
    joblib.dump(model, model_filename)

    # Save the updated custom dictionary
    new_vocabulary = set(vectorizer.get_feature_names_out())
    custom_dictionary.update(new_vocabulary)
    save_list_to_json(custom_dictionary_filename, custom_dictionary)

    # Save misclassified samples
    save_list_to_json(misclassified_samples_filename, misclassified_samples)

    # Check spelling and print corrections and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, corrected, suggestion_list in zip(words_to_check, corrected_words, suggestions):
        logging.info(f"Original: {word}, Corrected: {corrected}, Suggestions: {suggestion_list}")

    logging.info("Training loop finished.")
```

### Changes:

1. **Class Weighting**: I've added a function `train_model_with_class_weight()` that adjusts the model based on class distribution, which can help if your data is imbalanced.
2. **Logging Enhancements**: I've added better logging around cross-validation results and model evaluation.
3. **Performance Evaluation**: Consider integrating additional performance metrics to track your model's behavior across different class labels.

### Next Steps:

- Run the updated code and review the logs. Check for overfitting or performance degradation on the test set.
- Investigate using a more balanced dataset or applying data augmentation methods if your dataset is small or imbalanced.


In [23]:
import json
import logging
import os

import joblib
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils.class_weight import compute_class_weight
from spellchecker import SpellChecker

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


def load_labeled_data(filename):
    """Load labeled data from a JSON file and return it as a DataFrame."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Ensure 'data' and 'label' keys are present and non-empty in each item
        labeled_data = [
            (item["data"], item["label"])
            for item in data
            if "data" in item and "label" in item and item["data"] and item["label"]
        ]
        if not labeled_data:
            raise ValueError("No valid labeled data found.")
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except (FileNotFoundError, json.JSONDecodeError, ValueError) as e:
        logging.error(f"Error loading data: {e}")
        return pd.DataFrame(columns=["data", "labels"])


def load_list_from_json(filename):
    """Load a list of strings from a JSON file."""
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as file:
            return set(json.load(file))
    return set()


def save_list_to_json(filename, data_list):
    """Save a list of strings to a JSON file."""
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(list(data_list), file)


def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions for a list of words."""
    corrected_words = [spell.correction(word) for word in words_to_check]
    suggestions = [
        list(spell.candidates(word)) if spell.candidates(word) is not None else []
        for word in words_to_check
    ]
    return corrected_words, suggestions


def fit_vectorizer(X_train, vectorizer=None):
    """Fit or update a CountVectorizer for character n-grams."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)
    return vectorizer


def train_model_with_class_weight(X_train_vectorized, y_train):
    """Train a Naive Bayes model with class weights."""
    # Compute the class weights (this is an array of weights for each class)
    class_weights = compute_class_weight(
        "balanced", classes=np.unique(y_train), y=y_train
    )

    # Normalize the class weights to probabilities (they should sum to 1)
    class_probabilities = class_weights / class_weights.sum()

    # Pass the normalized class probabilities as class_prior
    model = MultinomialNB(class_prior=class_probabilities)
    model.fit(X_train_vectorized, y_train)

    return model


def main():
    logging.info("Starting the main process.")
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"
    model_filename = "naive_bayes_model.pkl"

    spell = SpellChecker(language="fr")

    # Load custom dictionary and misclassified samples as sets
    custom_dictionary = load_list_from_json(custom_dictionary_filename)
    spell.word_frequency.load_words(custom_dictionary)

    misclassified_samples = load_list_from_json(misclassified_samples_filename)

    misclassification_threshold = 5
    epochs = 0

    all_X_train = []  # Use list instead of pd.Series for efficient concatenation
    all_y_train = []

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if (
            labeled_df.empty
            or "labels" not in labeled_df.columns
            or "data" not in labeled_df.columns
        ):
            logging.error("Error: Labeled data is empty or missing required columns.")
            break

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train.extend(X_train)  # Append to the list
        all_y_train.extend(y_train)  # Append to the list

        vectorizer = fit_vectorizer(all_X_train)
        X_train_vectorized = vectorizer.transform(all_X_train)

        # Train model with class weights
        model = train_model_with_class_weight(X_train_vectorized, all_y_train)

        # Cross-validation
        n_splits = min(
            3,
            min(
                [
                    sum(pd.Series(all_y_train) == class_label)
                    for class_label in set(all_y_train)
                ]
            ),
        )
        if n_splits > 1:
            cv = StratifiedKFold(n_splits=n_splits)
            cv_scores = cross_val_score(model, X_train_vectorized, all_y_train, cv=cv)
            logging.info(
                f"Cross-validation scores: {cv_scores}, Mean score: {cv_scores.mean()}"
            )
        else:
            logging.warning("Not enough data to perform cross-validation.")

        # Check for misclassified samples
        words_to_check = ["motday", "hgost", "mney", "elphat", "rmshkana"]
        predictions = model.predict(vectorizer.transform(words_to_check))

        new_misclassified = [
            word
            for word, prediction in zip(words_to_check, predictions)
            if prediction == 0
        ]
        misclassified_samples.update(new_misclassified)

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save the trained model
    joblib.dump(model, model_filename)

    # Save the updated custom dictionary
    new_vocabulary = set(vectorizer.get_feature_names_out())
    custom_dictionary.update(new_vocabulary)
    save_list_to_json(custom_dictionary_filename, custom_dictionary)

    # Save misclassified samples
    save_list_to_json(misclassified_samples_filename, misclassified_samples)

    # Check spelling and print corrections and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, corrected, suggestion_list in zip(
        words_to_check, corrected_words, suggestions
    ):
        logging.info(
            f"Original: {word}, Corrected: {corrected}, Suggestions: {suggestion_list}"
        )

    logging.info("Training loop finished.")


if __name__ == "__main__":
    main()


2024-12-04 10:53:38,133 - INFO - Starting the main process.


2024-12-04 10:53:38,333 - WARNING - Not enough data to perform cross-validation.
2024-12-04 10:53:38,343 - INFO - Cross-validation scores: [1. 1.], Mean score: 1.0
2024-12-04 10:53:38,353 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:53:38,367 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:53:38,381 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:53:38,395 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:53:38,406 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:53:38,420 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:53:38,434 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:53:38,448 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:53:38,461 - INFO - Cross-validation scores: [1. 1. 1.], Mean score: 1.0
2024-12-04 10:53:38,482 - INFO - Cross-validation scores: [1. 